<a href="https://colab.research.google.com/github/Aafaaq77/machineLearningCoursera/blob/main/course2/REG02_NB01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create (numpy/pandas/sklearn) functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames (pandas instead) to do some feature engineering
* Use built-in Turi Create functions (numpy(pandas) to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

In [1]:
import numpy as np
import pandas as pd

### After importing the libraries, reading in the house data set.

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [4]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 
              'sqft_lot15':float, 'sqft_living':float, 'floors':str, 
              'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict, index_col=0)
sales.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


## Split data into training and testing.

In [5]:
# normally
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# but for the quiz I will be using the training and testing data provided
train_data = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict, index_col=0)
test_data = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict, index_col=0)

## Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

In [6]:
from sklearn.linear_model import LinearRegression

In [7]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = LinearRegression()

In [8]:
example_model.fit(train_data[example_features], train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [10]:
preds = example_model.predict(test_data[example_features])
example_model.coef_

array([   315.40669062, -65081.88711588,   6942.16598637])

In [11]:
preds[0]

350640.93803383986

## Compute RSS.

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [15]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    errors = outcome - predictions
    # Then square and add them up
    RSS = np.sum(errors*errors)
    return(RSS)  

In [16]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])
rss_example_train

273761940583133.75

## Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 

In [23]:
# bedrooms_squared = bedrooms*bedrooms
train_data['bedrooms_squared'] = train_data['bedrooms'] * train_data['bedrooms']
test_data['bedrooms_squared'] = test_data['bedrooms'] * test_data['bedrooms']

# bed_bath_rooms = bedrooms*bathrooms
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

# log_sqft_living = log(sqft_living)
train_data['log_sqft_living'] = np.log(train_data['sqft_living'])
test_data['log_sqft_living'] = np.log(test_data['sqft_living'])

# lat_plus_long = lat + long
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

In [22]:
print(log(3))
print(np.log(3))

1.0986122886681098
1.0986122886681098


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [24]:
print('mean of bedrooms_squared: ', test_data['bedrooms_squared'].mean())
print('mean of bed_bathrooms: ', test_data['bed_bath_rooms'].mean())
print('mean of log_sqft_living: ', test_data['log_sqft_living'].mean())
print('mean of lat+long: ', test_data['lat_plus_long'].mean())

mean of bedrooms_squared:  12.4466777015843
mean of bed_bathrooms:  7.5039016315913925
mean of log_sqft_living:  7.550274679645921
mean of lat+long:  -74.65333355403185


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [25]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

In [26]:
model_1 = LinearRegression()
model_2 = LinearRegression()
model_3 = LinearRegression()

In [27]:
model_1.fit(train_data[model_1_features], train_data['price'])
model_2.fit(train_data[model_2_features], train_data['price'])
model_3.fit(train_data[model_3_features], train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

In [28]:
model_1.coef_

array([ 3.12258646e+02, -5.95865332e+04,  1.57067421e+04,  6.58619264e+05,
       -3.09374351e+05])

In [29]:
model_2.coef_[2]

-71461.30829275973

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [31]:
get_residual_sum_of_squares(model_1, train_data[model_1_features], train_data['price'])

967879963049545.8

In [32]:
get_residual_sum_of_squares(model_2, train_data[model_2_features], train_data['price'])

958419635074068.8

In [33]:
get_residual_sum_of_squares(model_3, train_data[model_3_features], train_data['price'])

903436455050478.0

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

In [34]:
get_residual_sum_of_squares(model_3, train_data[model_3_features], train_data['price']) < get_residual_sum_of_squares(model_2, train_data[model_2_features], train_data['price'])

True

Now compute the RSS on on TEST data for each of the three models.

In [35]:
get_residual_sum_of_squares(model_1, test_data[model_1_features], test_data['price'])

225500469795489.56

In [36]:
get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data['price'])

223377462976467.16

In [37]:
get_residual_sum_of_squares(model_3, test_data[model_3_features], test_data['price'])

259236319207179.62

Quiz Question: Which model (1, 2, or 3) had the lowest RSS on TESTING data?

In [38]:
get_residual_sum_of_squares(model_3, test_data[model_3_features], test_data['price']) < get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data['price'])

False

In [39]:
get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data['price']) < get_residual_sum_of_squares(model_1, test_data[model_1_features], test_data['price'])

True